Antonio Merino Gallardo

# Códigos Goppa Binarios

Los códigos Goppa constituyen una importante familia de códigos lineales. Nos centraremos en el caso binario.

Dado un polinomio $g(x) \in \mathbb{F}_{2^m}[x]$ consideramos el anillo cociente

$$S_m = \frac{\mathbb{F}_{2^m}[x]}{\langle G(x) \rangle}.$$

En este anillo definimos

$$\frac{1}{x-\alpha}:=-\frac{G(x)-G(\alpha)}{x-\alpha}G(\alpha)^{-1}.$$

Tomemos ahora un conjunto $L = \{\alpha_1,\alpha_2,...,\alpha_n\} \subseteq \mathbb{F}_{2^m}$. Dada una palabra $a=a_1a_2\cdots a_n \in \mathbb{F}_{2}^n$ definimos

$$R_a(x):=\sum_{i=1}^{n}\frac{a_i}{x-\alpha_i} \in S_m.$$

De este módo, el código Goppa binario $\Gamma(L,g)$ se define como

$$\Gamma(L,g)=\{a \in \mathbb{F}_{2}^n | R_a(x) \equiv 0 \; (mod \; g(x))\}.$$

Con la clase CodigoGoppaBinario implementada, podemos construir un código Goppa binario a partir de su polinomio generador y su conjunto de definición.

In [1]:
from codigo_goppa_binario import CodigoGoppaBinario

In [2]:
F= GF(2^3)
R.<x> = F[]
g = x^2 + 1
L = [a for a in F.list() if g(a) != 0]
C = CodigoGoppaBinario(g, L)
C

[7, 4] Código Goppa Binario

### Matriz de paridad

Al tratarse de códigos lineales, admiten también una matriz de paridad.

Si el polinomio generador se expresa como $g(x)=\sum_{k=0}^{t}g_kx^k$, la matriz que buscamos es

$$H = \begin{pmatrix}
    g_tg(\alpha_1)^{-1} & \dots & g_tg(\alpha_n)^{-1}\\
    (g_{t-1}+g_t\alpha_1)g(\alpha_1)^{-1} & \dots & (g_{t-1}+g_t\alpha_n)g(\alpha_n)^{-1}\\
    \vdots & \ddots & \vdots\\
    \left(\sum_{j=1}^tg_j\alpha_1^{j-1}\right)g(\alpha_1)^{-1} & \dots & \left(\sum_{j=1}^tg_j\alpha_n^{j-1}\right)g(\alpha_n)^{-1}\\
\end{pmatrix}.$$ 

Esta matriz verifica que $a\in \Gamma(L,G)$ si, y solo si, $Ha^T=0.$ Sin embargo, no es una matriz de paridad en el sentido estricto. Por un lado, podría tener filas linealmente dependientes, que se podrían eliminar para que tuviera rango máximo. Por otro lado, sus entradas son elementos de $\mathbb{F}_{2^m}$ y no de $\mathbb{F}_2$. 

Si considermos los elementos de $\mathbb{F}_{2^m}$ como vectores columna en $\mathbb{F}_{2}^m$, podemos extender la matriz $H$ de dimensión $(t \times n)$ a una matriz $H'$ de dimensión $(tm \times n)$ con entradas en $\mathbb{F}_2$.

La matriz $H$ descrita en primer lugar es la que nos devuelve el método *matriz_paridad*, mientras que el método *matriz_paridad_extendida* nos devuelve la matriz $H'$.

In [3]:
C.matriz_paridad()

[            1            z3          z3^2 z3^2 + z3 + 1     z3^2 + z3      z3^2 + 1        z3 + 1]
[            0          z3^2     z3^2 + z3            z3            z3     z3^2 + z3          z3^2]

In [4]:
C.matriz_paridad_extendida()

[1 0 0 1 0 1 1]
[0 1 0 1 1 0 1]
[0 0 1 1 1 1 0]
[0 0 0 0 0 0 0]
[0 0 1 1 1 1 0]
[0 1 1 0 0 1 1]

A partir de una matriz de paridad, se puede calcular una matriz generadora teniendo en cuenta la dualidad de los códigos que generan. Esta matriz generadora la podemos obtener con el método *matriz_generadora*.

In [5]:
C.matriz_generadora()

[1 0 0 1 0 1 1]
[0 1 0 1 0 1 0]
[0 0 1 1 0 0 1]
[0 0 0 0 1 1 1]

Cabe notar que en el cálculo de esta matriz sí se obtienen filas linealmente independientes, luego podemos conocer así la dimensión del código, que coincide con el número de filas de esta matriz.

In [6]:
C.dimension()

4

### Codificación y decodificación

El proceso de codificación es sencillo, pues es el que se emplea en todo código lineal, consistente en multiplicar la palabra a codificar por la matriz generadora.

In [7]:
w = vector(GF(2), [0,1,0,1]); w

(0, 1, 0, 1)

In [8]:
w*C.matriz_generadora()

(0, 1, 0, 1, 1, 0, 1)

In [9]:
y = C.codificar(w); y

(0, 1, 0, 1, 1, 0, 1)

Por otro lado, la tarea de decodificación, entendida como encontrar la palabra código más cercana a una palabra dada, se ha implementado mediante el algoritmo de Patterson.

Definimos un patrón de error y se lo sumamos a y.

In [10]:
e = vector(GF(2), [0,1,0,0,0,0,0]); e

(0, 1, 0, 0, 0, 0, 0)

In [11]:
y2 = y + e; y2

(0, 0, 0, 1, 1, 0, 1)

In [12]:
C.decodificar(y2)

(0, 1, 0, 1, 1, 0, 1)

In [13]:
C.decodificar(y2) == y

True

### Códigos Goppa Binarios Irreducibles

Un código Goppa Binario es irreducible cuando su polinomio generador lo es. En este caso, la distancia mínima $d$ del código verifica $d \geq 2t+1$, donde $t$ es el grado del polinomio generador. De este modo, estos códigos son capaces de corregir al menos $t$ errores.

Construyamos entonces un código Goppa binario a partir de un polinomio irreducible de grado $4$ y veamos cómo podemos corregir 4 errores.

Definimos primero el cuerpo base del código.

In [14]:
F= GF(2^7)
R.<x> = F[]

Obtenemos ahora un polinomio irreducible generando aleatoriamente polinomios de grado 4.

In [15]:
set_random_seed(0)

In [16]:
g = R(0)
while not g.is_irreducible():
    g = R.random_element(4)
g = g/g.list()[len(g.list())-1]
g

x^4 + (z7^6 + z7^3 + z7)*x^3 + (z7^6 + z7^5 + z7^3 + z7^2)*x^2 + (z7^5 + z7^3 + z7 + 1)*x + z7^5 + z7^2 + z7

Construimos ahora el código, pudiendo tomar como conjunto de definición $L = \mathbb{F}_{2^7}$ por ser $g$ irreducible.

In [17]:
L = F.list()
C = CodigoGoppaBinario(g, L)
C

[128, 100] Código Goppa Binario

Podemos obtener sus matrices de paridad y generadora.

In [18]:
C.matriz_paridad()

4 x 128 dense matrix over Finite Field in z7 of size 2^7 (use the '.str()' method to see the entries)

In [19]:
H = C.matriz_paridad_extendida(); H

28 x 128 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)

In [20]:
G = C.matriz_generadora(); G

100 x 128 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)

In [21]:
(G*H.transpose()).is_zero()

True

Por otro lado, podemos codificar una palabra de $\mathbb{F}_2^{100}$.

In [22]:
x = VectorSpace(GF(2), 100).random_element(); x

(0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0)

In [23]:
y = C.codificar(x); y

(0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1)

Definamos entonces un patrón de error de peso $4$ y veamos como la decodificación es capaz de corregir los errores.

In [24]:
e = vector(GF(2), 128*[0])
e[4] = 1
e[5] = 1
e[7] = 1
e[100] = 1
e

(0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

In [25]:
y2 = y + e; y2

(0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1)

In [26]:
y3 = C.decodificar(y2); y3

(0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1)

In [27]:
y3 == y

True